In [ ]:
from scipy.optimize import linprog
import numpy as np

# Debt data: (Loan amount, Interest rate, Min payment)
debts = np.array([
    [5000, 0.15, 200],  # Loan 1: M5,000 at 15% interest, min payment M200
    [3000, 0.12, 150],  # Loan 2: M3,000 at 12% interest, min payment M150
])

# Monthly income & expenses
income = 5000  
expenses = 2500  

# Define LP problem
num_debts = len(debts)
c = debts[:, 1]  # Interest rates (minimize total interest)
A = np.vstack([-np.eye(num_debts), np.ones(num_debts)])  # Constraints for debt payment
b = np.hstack([-debts[:, 2], income - expenses])  # Payments should cover min requirements

# Solve the LP problem
result = linprog(c, A_ub=A, b_ub=b, method="highs")
optimal_payments = result.x  

print("Optimized Debt Payments:", optimal_payments)


In [7]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np

# Sample Data
data = pd.DataFrame({
    "income": [5000, 6000, 4000, 3000, 7000],
    "expenses": [2500, 3000, 2200, 2700, 3500],
    "debt": [5000, 2000, 7000, 1000, 4000],
    "savings": [500, 1000, 300, 200, 1200],
    "label": ["Stable", "Stable", "At Risk", "Critical", "Stable"]
})

# Split features & labels
X = data.drop(columns=["label"])
y = data["label"]

# Train Decision Tree
model = DecisionTreeClassifier()
model.fit(X, y)

# Predict Financial Status
new_user = np.array([[3000, 3200, 2500, 800]])  # Sample user data
prediction = model.predict(new_user)
print("Financial Status:", prediction)


Financial Status: ['Critical']


c:\Users\Administrator\Desktop\projects\thabo motlakase\.venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


In [5]:
import numpy as np
from scipy.optimize import linprog

# Debt Data: (Loan amount, Interest rate (monthly), Min payment, Max payment)
debts = np.array([
    [5000, 0.15/12, 200, 1000],  # Loan 1: M5,000 at 15% annual interest
    [3000, 0.12/12, 150, 700],   # Loan 2: M3,000 at 12% annual interest
])

# User's financial profile
income_variability = [5000, 4000, 5500, 6000, 4500, 5000, 4800, 5200, 5300, 4900, 4700, 5100]  # Monthly income
expenses = 2500  # Fixed monthly expenses
months = 12  # Planning period
num_debts = len(debts)

# Initialize payment matrix (rows: months, cols: debts)
payments = np.zeros((months, num_debts))
remaining_balances = debts[:, 0].copy()

def prioritize_debts(strategy="avalanche"):
    """Returns the sorted index of debts based on the chosen strategy."""
    if strategy == "avalanche":  # Highest interest rate first
        return np.argsort(-debts[:, 1])
    elif strategy == "snowball":  # Lowest balance first
        return np.argsort(debts[:, 0])
    return np.arange(num_debts)

priority_order = prioritize_debts(strategy="avalanche")  # Change to "snowball" if needed

for month in range(months):
    available_cash = income_variability[month] - expenses  # Calculate available cash
    c = debts[:, 1]  # Minimize total interest
    A = np.vstack([-np.eye(num_debts), np.ones(num_debts)])  # Payment constraints
    b = np.hstack([-debts[:, 2], available_cash])  # Min payment & budget limit
    
    # Solve LP for optimal payments
    result = linprog(c, A_ub=A, b_ub=b, bounds=[(debts[i, 2], debts[i, 3]) for i in range(num_debts)], method="highs")
    payments[month, :] = result.x if result.success else debts[:, 2]  # Use min payments if LP fails
    
    # Update remaining balances & compound interest for unpaid balances
    for i in priority_order:
        remaining_balances[i] = max(0, remaining_balances[i] - payments[month, i])
        remaining_balances[i] += remaining_balances[i] * debts[i, 1]  # Apply interest

# Print final optimized payments
print("\nOptimized Monthly Payments:")
print(payments)
print("\nRemaining Balances After", months, "Months:")
print(remaining_balances)



Optimized Monthly Payments:
[[200. 150.]
 [200. 150.]
 [200. 150.]
 [200. 150.]
 [200. 150.]
 [200. 150.]
 [200. 150.]
 [200. 150.]
 [200. 150.]
 [200. 150.]
 [200. 150.]
 [200. 150.]]

Remaining Balances After 12 Months:
[3199.5494015 1459.0758839]


In [8]:
import numpy as np
from scipy.optimize import linprog

# User's financial profile
income_variability = [5000, 4000, 5500, 6000, 4500, 5000, 4800, 5200, 5300, 4900, 4700, 5100]  # Monthly income
expenses = 2500  # Fixed monthly expenses
months = 12  # Planning period

# Loan options: (Max Loan Amount, Interest Rate (Annual), Min Payment, Max Payment)
loan_options = np.array([
    [5000, 0.15, 200, 1000],  # Loan 1: M5,000 at 15% annual interest
    [3000, 0.12, 150, 700],   # Loan 2: M3,000 at 12% annual interest
    [7000, 0.18, 300, 1200],  # Loan 3: M7,000 at 18% annual interest
    [4000, 0.10, 180, 800],   # Loan 4: M4,000 at 10% annual interest
])

# Convert annual interest to monthly interest
loan_options[:, 1] /= 12

# Determine the best loan based on available cash flow
available_cash = np.mean(income_variability) - expenses

def recommend_loan():
    best_loan = None
    best_score = float('inf')
    
    for loan in loan_options:
        max_affordable_payment = min(available_cash, loan[3])
        if max_affordable_payment < loan[2]:
            continue  # Skip unaffordable loans
        
        total_interest = loan[1] * loan[0] * months  # Approximate total interest over period
        score = total_interest / max_affordable_payment  # Lower is better
        
        if score < best_score:
            best_score = score
            best_loan = loan
    
    return best_loan

recommended_loan = recommend_loan()
print("\nRecommended Loan Based on Income & Expenses:")
print(f"Amount: M{recommended_loan[0]:.2f}, Interest Rate: {recommended_loan[1] * 12:.2%}, Min Payment: M{recommended_loan[2]:.2f}, Max Payment: M{recommended_loan[3]:.2f}")

# Initialize payment matrix (rows: months, cols: debts)
payments = np.zeros((months, 1))
remaining_balance = recommended_loan[0]
interest_rate = recommended_loan[1]

for month in range(months):
    payment = min(available_cash, recommended_loan[3])  # Pay as much as possible
    payments[month, 0] = payment
    remaining_balance = max(0, remaining_balance - payment)
    remaining_balance += remaining_balance * interest_rate  # Apply interest

print("\nOptimized Monthly Payments:")
print(payments)
print("\nRemaining Balance After", months, "Months:")
print(remaining_balance)


Recommended Loan Based on Income & Expenses:
Amount: M4000.00, Interest Rate: 10.00%, Min Payment: M180.00, Max Payment: M800.00

Optimized Monthly Payments:
[[800.]
 [800.]
 [800.]
 [800.]
 [800.]
 [800.]
 [800.]
 [800.]
 [800.]
 [800.]
 [800.]
 [800.]]

Remaining Balance After 12 Months:
0.0


In [10]:
import numpy as np
from scipy.optimize import linprog

# User's financial profile
income_variability = [5000, 4000, 5500, 6000, 4500, 5000, 4800, 5200, 5300, 4900, 4700, 5100]  # Monthly income
expenses_variability = [2500, 2600, 2400, 2300, 2700, 2500, 2550, 2450, 2600, 2550, 2500, 2400]  # Monthly expenses
months = 12  # Planning period

# Loan options: (Max Loan Amount, Interest Rate (Annual), Min Payment, Max Payment)
loan_options = np.array([
    [5000, 0.10, 200, 1000],  # Loan 1: M5,000 at 15% annual interest
    [3000, 0.30, 150, 700],   # Loan 2: M3,000 at 12% annual interest
    [7000, 0.20, 300, 1200],  # Loan 3: M7,000 at 18% annual interest
    [4000, 0.25, 180, 800],   # Loan 4: M4,000 at 10% annual interest
])

# Convert annual interest to monthly interest
loan_options[:, 1] /= 12

# Determine the best loan based on available cash flow
average_income = np.mean(income_variability)
average_expenses = np.mean(expenses_variability)
available_cash = average_income - average_expenses

def recommend_loan():
    best_loan = None
    best_score = float('inf')
    
    for loan in loan_options:
        max_affordable_payment = min(available_cash, loan[3])
        if max_affordable_payment < loan[2]:
            continue  # Skip unaffordable loans
        
        total_interest = loan[1] * loan[0] * months  # Approximate total interest over period
        score = total_interest / max_affordable_payment  # Lower is better
        
        if score < best_score:
            best_score = score
            best_loan = loan
    
    return best_loan

recommended_loan = recommend_loan()
print("\nRecommended Loan Based on Income & Expenses:")
print(f"Amount: M{recommended_loan[0]:.2f}, Interest Rate: {recommended_loan[1] * 12:.2%}, Min Payment: M{recommended_loan[2]:.2f}, Max Payment: M{recommended_loan[3]:.2f}")

# Initialize payment matrix (rows: months, cols: debts)
payments = np.zeros((months, 1))
remaining_balance = recommended_loan[0]
interest_rate = recommended_loan[1]

for month in range(months):
    available_cash = income_variability[month] - expenses_variability[month]  # Adjust per month
    payment = min(available_cash, recommended_loan[3])  # Pay as much as possible
    payments[month, 0] = payment
    remaining_balance = max(0, remaining_balance - payment)
    remaining_balance += remaining_balance * interest_rate  # Apply interest

print("\nOptimized Monthly Payments:")
print(payments)
print("\nRemaining Balance After", months, "Months:")
print(remaining_balance)



Recommended Loan Based on Income & Expenses:
Amount: M5000.00, Interest Rate: 10.00%, Min Payment: M200.00, Max Payment: M1000.00

Optimized Monthly Payments:
[[1000.]
 [1000.]
 [1000.]
 [1000.]
 [1000.]
 [1000.]
 [1000.]
 [1000.]
 [1000.]
 [1000.]
 [1000.]
 [1000.]]

Remaining Balance After 12 Months:
0.0


In [11]:
import numpy as np

# User's financial profile
income_variability = [5000, 4000, 5500, 6000, 4500, 5000, 4800, 5200, 5300, 4900, 4700, 5100]  # Monthly income
expenses_variability = [2500, 2600, 2400, 2300, 2700, 2500, 2550, 2450, 2600, 2550, 2500, 2400]  # Monthly expenses
months = 12  # Planning period

# Loan options: (Max Loan Amount, Interest Rate (Annual), Min Payment, Max Payment)
loan_options = np.array([
    [5000, 0.15, 200, 1000],  # Loan 1: M5,000 at 15% annual interest
    [3000, 0.12, 150, 700],   # Loan 2: M3,000 at 12% annual interest
    [7000, 0.18, 300, 1200],  # Loan 3: M7,000 at 18% annual interest
    [4000, 0.10, 180, 800],   # Loan 4: M4,000 at 10% annual interest
])

# Convert annual interest to monthly interest
loan_options[:, 1] /= 12

# Determine the best loan based on available cash flow
average_income = np.mean(income_variability)
average_expenses = np.mean(expenses_variability)
available_cash = average_income - average_expenses

def recommend_loan():
    best_loan = None
    best_amount = 0
    best_months = 0
    best_score = float('inf')
    
    for loan in loan_options:
        max_affordable_payment = min(available_cash, loan[3])
        if max_affordable_payment < loan[2]:
            continue  # Skip unaffordable loans
        
        for term in range(6, 61, 6):  # Loan terms from 6 to 60 months
            monthly_interest = loan[1]
            loan_amount = min(loan[0], max_affordable_payment * term)  # Determine max loan amount
            
            total_interest = 0
            balance = loan_amount
            for _ in range(term):
                interest = balance * monthly_interest
                total_interest += interest
                balance -= max_affordable_payment
                balance += interest  # Add interest to balance
                if balance <= 0:
                    break
            
            score = total_interest / term  # Lower is better
            
            if score < best_score:
                best_score = score
                best_loan = loan
                best_amount = loan_amount
                best_months = term
    
    return best_loan, best_amount, best_months

recommended_loan, recommended_amount, recommended_months = recommend_loan()

print("\nRecommended Loan Based on Income & Expenses:")
print(f"Amount: M{recommended_amount:.2f}, Interest Rate: {recommended_loan[1] * 12:.2%}, Min Payment: M{recommended_loan[2]:.2f}, Max Payment: M{recommended_loan[3]:.2f}, Repayment Period: {recommended_months} months")

# Initialize payment matrix (rows: months, cols: debts)
payments = np.zeros((recommended_months, 1))
remaining_balance = recommended_amount
interest_rate = recommended_loan[1]

for month in range(recommended_months):
    available_cash = income_variability[month % len(income_variability)] - expenses_variability[month % len(expenses_variability)]  # Adjust per month
    payment = min(available_cash, recommended_loan[3])  # Pay as much as possible
    payments[month, 0] = payment
    remaining_balance = max(0, remaining_balance - payment)
    remaining_balance += remaining_balance * interest_rate  # Apply interest

print("\nOptimized Monthly Payments:")
print(payments)
print("\nRemaining Balance After", recommended_months, "Months:")
print(remaining_balance)


Recommended Loan Based on Income & Expenses:
Amount: M3000.00, Interest Rate: 12.00%, Min Payment: M150.00, Max Payment: M700.00, Repayment Period: 60 months

Optimized Monthly Payments:
[[700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]
 [700.]]

Remaining Balance After 60 Months:
0.0


In [ ]:
import numpy as np

# ========== User Inputs ==========
# Cash flow history (monthly figures)
income_history = np.array([5000, 4000, 5500, 6000, 4500, 5000, 4800, 5200, 5300, 4900, 4700, 5100])
expense_history = np.array([2500, 2600, 2400, 2300, 2700, 2500, 2550, 2450, 2600, 2550, 2500, 2400])
months_history = len(income_history)

# Target savings (monthly)
target_savings = 800  # e.g., M800 saved per month

# Assets: investments, properties, etc.
# You might have a structured dictionary of asset types and values.
assets = {
    'investments': 15000,
    'property': 50000
}
total_assets = sum(assets.values())

# Collateral factor (e.g., you decide you want the maximum loan to be 50% of total assets)
collateral_factor = 0.5
max_loan_by_assets = total_assets * collateral_factor

# ========== Loan Option Data ==========
# Each row: [Min Loan, Max Loan, Annual Interest, Minimum Monthly Payment, Maximum Monthly Payment]
loan_options = np.array([
    [1000, 5000, 0.15, 200, 1000],
    [800, 3000, 0.12, 150, 700],
    [2000, 7000, 0.18, 300, 1200],
    [500, 4000, 0.10, 180, 800],
], dtype=float)

# Convert annual interest to monthly interest for each option:
loan_options[:, 2] /= 12

# ========== Step 1: Determine Financial Flexibility ==========
# Compute average income and expenses from historical data
avg_income = np.mean(income_history)
avg_expenses = np.mean(expense_history)
avg_discretionary = avg_income - avg_expenses - target_savings

print("Average Income: M{:.2f}".format(avg_income))
print("Average Expenses: M{:.2f}".format(avg_expenses))
print("Target Savings: M{:.2f}".format(target_savings))
print("Available monthly surplus for loan repayment: M{:.2f}".format(avg_discretionary))

# Advise whether a loan is even necessary.
if avg_discretionary <= 0:
    print("\nBased on your cash flow, taking on additional debt may strain your ability to meet your savings targets.")
    advise = "Avoid taking a new loan; consider cutting expenses or increasing your income."
else:
    advise = "A loan might be feasible given your surplus cash flow."
print("\nAdvice on taking a loan:")
print(advise)

# ========== Step 2: Determine Safe Loan Range ==========
# Define safe loan range:
#   - Lower bound: the minimum that would make a difference (could be based on specific needs)
#   - Upper bound: the minimum of the maximum allowed by the loan option and what your assets cover
safe_loan_lower = 0  # Possibly, if you need any extra working capital, determine a minimum required amount
safe_loan_upper = max_loan_by_assets
print("\nBased on your assets, you can safely collateralize up to: M{:.2f}".format(safe_loan_upper))

# ========== Step 3: Loan Option Simulation ==========
# For each loan option, and for a range of terms and loan amounts, simulate the monthly repayment schedule.
def simulate_repayment(principal, monthly_rate, term, monthly_payment):
    """
    Simulate the amortization of a loan given:
      - principal: loan amount
      - monthly_rate: monthly interest rate
      - term: number of months over which repayment is made
      - monthly_payment: fixed monthly payment amount
    Returns total interest paid and final balance (if any)
    """
    balance = principal
    total_interest = 0.0
    for _ in range(term):
        interest = balance * monthly_rate
        total_interest += interest
        balance = balance + interest - monthly_payment
        if balance <= 0:
            break
    return total_interest, balance

# We want to choose the option that minimizes a penalty function such as:
#    penalty = (total_interest/term) + lambda * (shortfall in maintaining target surplus)
# where lambda is a factor that weighs the importance of saving targets.
lambda_penalty = 1.0  # adjust based on risk tolerance and target importance

results = []
term_range = range(6, 61, 6)  # consider terms from 6 months to 60 months

# For each loan option, simulate over a grid of principal amounts and terms.
for idx, option in enumerate(loan_options):
    min_loan, max_loan, monthly_rate, min_payment, max_payment = option
    # Adjust maximum possible loan for this option by both the product constraints and your collateral
    option_max = min(max_loan, safe_loan_upper)
    # Use a small grid for loan amounts: from min_loan to option_max (if option_max is higher than min_loan)
    if option_max < min_loan:
        continue  # Skip options that you cannot use because the upper bound is below the lender's minimum.
    principal_grid = np.linspace(min_loan, option_max, num=5)  # 5 sample points
    for term in term_range:
        for principal in principal_grid:
            # Determine what monthly payment can be "afforded" given your surplus and the option limits.
            # Here, we assume you want to use up to your available surplus but not exceeding the max allowed by the loan.
            feasible_payment = min(avg_discretionary, max_payment)
            # But the payment must at least be the lender's minimum requirement.
            if feasible_payment < min_payment:
                continue  # if the payment isn't high enough to satisfy the minimum payment requirement.
            total_interest, final_balance = simulate_repayment(principal, monthly_rate, term, feasible_payment)
            # Compute an adjustment penalty if the payment structure forces you to dip into your savings target.
            # For simulation purposes, if the feasible payment is below what is required to amortize the loan,
            # you might “penalize” that option.
            if final_balance > 0:
                penalty = 1000  # heavy penalty if the loan would not be paid off in term
            else:
                penalty = 0
            score = (total_interest/term) + lambda_penalty * penalty
            results.append({
                "option_index": idx,
                "loan_parameters": option,
                "term": term,
                "principal": principal,
                "monthly_payment": feasible_payment,
                "avg_interest_per_month": total_interest/term,
                "score": score
            })

# ========== Step 4: Selection & Recommendation ==========
if not results:
    print("\nNo loan option meets the feasibility criteria based on your current financial profile.")
else:
    # Choose the result with the lowest score.
    best_result = min(results, key=lambda x: x["score"])
    print("\n--- Loan Optimization Recommendation ---")
    option_used = loan_options[int(best_result["option_index"])]
    print("Recommended Loan Option:")
    print(" - Loan Details:")
    print("     * Minimum Loan: M{:.2f}, Maximum Loan: M{:.2f}".format(option_used[0], option_used[1]))
    print("     * Annual Interest Rate: {:.2%}".format(option_used[2] * 12))
    print("     * Payment Range: M{:.2f} - M{:.2f} per month".format(option_used[3], option_used[4]))
    print(" - Recommended Principal: M{:.2f}".format(best_result["principal"]))
    print(" - Recommended Term: {} months".format(best_result["term"]))
    print(" - Estimated Average Interest Cost per Month: M{:.2f}".format(best_result["avg_interest_per_month"]))
    print(" - Monthly Payment (affordable): M{:.2f}".format(best_result["monthly_payment"]))
    print("\nNote: The simulation also takes into account your target saving requirement and the asset-based collateral constraints.")


Average Income: M5000.00
Average Expenses: M2504.17
Target Savings: M800.00
Available monthly surplus for loan repayment: M1695.83

Advice on taking a loan:
A loan might be feasible given your surplus cash flow.

Based on your assets, you can safely collateralize up to: M32500.00

--- Loan Optimization Recommendation ---
Recommended Loan Option:
 - Loan Details:
     * Minimum Loan: M500.00, Maximum Loan: M4000.00
     * Annual Interest Rate: 10.00%
     * Payment Range: M180.00 - M800.00 per month
 - Recommended Principal: M500.00
 - Recommended Term: 60 months
 - Estimated Average Interest Cost per Month: M0.07
 - Monthly Payment (affordable): M800.00

Note: The simulation also takes into account your target saving requirement and the asset-based collateral constraints.
